In [ ]:
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
from sklego.meta.estimator_transformer import EstimatorTransformer
from scores.probability import crps_cdf
import pandas as pd
from functools import partial
from scipy import stats

from src.marginal_bootstrap import *

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sklearn.set_config(enable_metadata_routing=True)

In [ ]:
plt.style.use("bmh")

In [ ]:
SEED = 42
SAMPLE = 200
BOOTSTRAPS_OUTER = 1000
BOOTSTRAPS_INNER = 500

RUNS = 1500

statistic_of_interest = partial(np.mean, axis=0)

distribution = stats.beta(9, 1)

TRUE_MEAN = distribution.stats()[0]

random_generator = np.random.default_rng(SEED)
sample = distribution.rvs(SAMPLE, random_state=random_generator)

In [ ]:
sns.histplot(sample)
plt.axvline(TRUE_MEAN, color="firebrick", linestyle="--", label=f"Mean={TRUE_MEAN:.3f}")
plt.legend();

In [ ]:
bootstrap_methods = {
    "bayesian": bootstrap_bayesian,
    "jitter": bootstrap_with_jitter,
    "non_parametric": bootstrap_non_parametric,
    "kde": bootstrap_kde,
}

In [ ]:
runs = []
for method_name, method in tqdm(bootstrap_methods.items(), desc="Outer Loop", position=0):
    outer_bootstraps = bootstrap_non_parametric(sample, bootstrap_size=BOOTSTRAPS_OUTER, random_generator=random_generator)
    for inner_index, outer_bootstrap in tqdm(enumerate(outer_bootstraps.T), total=BOOTSTRAPS_OUTER, desc=f"Inner Loop - {method_name}"):
        inner_bootstrap = method(outer_bootstrap, bootstrap_size=BOOTSTRAPS_INNER, random_generator=random_generator)
        result = inner_index, method_name, statistic_of_interest(inner_bootstrap)
        runs.append(result)

In [ ]:
df = pd.DataFrame(runs, columns=["outer_index", "method", "means"]).explode("means")
df

In [ ]:
summary_df = (
    df.groupby(["method", "outer_index"])
    .agg(
        lower=("means", partial(np.quantile, q=0.025)),
        upper=("means", partial(np.quantile, q=0.975)),
    )
    .assign(
        width=lambda x: x.upper - x.lower,
        midpoint=lambda x: (x.upper + x.lower) / 2,
    )
    .reset_index(drop=False)
)
summary_df

In [ ]:
def coverage_ratio(true_value, lower, upper):
    return np.mean((true_value >= lower) & (true_value <= upper))

In [ ]:
def regression_mwi_score(
    y_true,
    lower,
    upper,
    confidence_level: float = 0.95
) -> float:

    y_pis = np.stack([lower, upper]).T[:, :, np.newaxis]
    y_pred_low = np.minimum(y_pis[:, 0, 0], y_pis[:, 1, 0])
    y_pred_up = np.maximum(y_pis[:, 0, 0], y_pis[:, 1, 0])

    width = np.sum(y_pred_up) - np.sum(y_pred_low)  # type: ignore
    error_above = np.sum((y_true - y_pred_up)[y_true > y_pred_up])
    error_below = np.sum((y_pred_low - y_true)[y_true < y_pred_low])
    total_error = error_above + error_below
    mwi = (width + total_error * 2 / (1 - confidence_level)) / len(lower)
    return mwi

In [ ]:
def isd_from_kde(samples, grid_size=20):
    kde = gaussian_kde(samples)
    x = np.linspace(min(samples), max(samples), grid_size)
    dx = x[1] - x[0]
    f_vals = kde(x)
    
    second_deriv = np.gradient(np.gradient(f_vals, dx), dx)
    
    isd = np.sum(second_deriv**2) * dx

    return isd

def entropy(samples, grid_size=20):
    kde = gaussian_kde(samples)
    x = np.linspace(min(samples), max(samples), grid_size)
    dx = x[1] - x[0]
    f_vals = kde(x)

    f_norm = f_vals / np.sum(f_vals * dx) 
    return -np.sum(f_norm * np.log(f_norm + 1e-12)) * dx

In [ ]:
from scipy import stats
from scores.probability import crps_cdf
import xarray

def cprs(true_value, samples, grid_size=20):
    x = np.linspace(min(samples), max(samples), grid_size)
    fcst_cdf = stats.norm.cdf(x, loc=0)
    fcst_array = xarray.DataArray(coords={'temperature': x}, data=fcst_cdf)
    obs_array = xarray.DataArray(true_value)
    return crps_cdf(fcst_array, obs_array, threshold_dim='temperature').total.values.round(3)

In [ ]:
runs_df = pd.concat([
    summary_df.groupby("method").sample(frac=1, replace=True, random_state=boot).assign(run=boot)
    for boot in range(RUNS)
])
runs_df

In [ ]:
aggregated_metrics = {
    "coverage_ratio": lambda x: coverage_ratio(TRUE_MEAN, x.lower, x.upper),
    "mwi": lambda x: regression_mwi_score(TRUE_MEAN, x.lower, x.upper),
    "width_mean": lambda x: np.mean(x.upper - x.lower),
    "lower_std": lambda x: np.std(x.lower),
    "upper_std": lambda x: np.std(x.upper),
    # "entropy_width": lambda x: entropy(x.width),
    # "entropy_midpoint": lambda x: entropy(x.midpoint),
    "bias_midpoint": lambda x: np.mean(x.midpoint) - TRUE_MEAN,
}

grouped_runs = runs_df.groupby(["method", "run"])

aggregated_data = {}
for metric_name, metric in aggregated_metrics.items():
    aggregated_data[metric_name] = grouped_runs.apply(metric)

In [ ]:
aggregated_df = pd.concat([
    dataframe.to_frame("value").assign(metric=metric_name)
    for metric_name, dataframe in aggregated_data.items()
]).reset_index()

In [ ]:
data_metrics = {
    "log(isd)": lambda x: np.log(isd_from_kde(x.means.astype(np.float64))),
    "cprs": lambda x: cprs(TRUE_MEAN, x.means.astype(np.float64)),
    "bias_point": lambda x: np.mean(x.means) - TRUE_MEAN
                        
}

df_grouped = df.groupby(["method", "outer_index"])

data_aggregated_data = {}
for metric_name, metric in data_metrics.items():
    data_aggregated_data[metric_name] = df_grouped.apply(metric)

In [ ]:
data_aggregated_df = pd.concat([
    dataframe.to_frame("value").assign(metric=metric_name)
    for metric_name, dataframe in data_aggregated_data.items()
]).reset_index()

In [ ]:
study_summary_df = pd.concat([aggregated_df, data_aggregated_df]).drop(["run", "outer_index"], axis=1)
study_summary_df

In [ ]:
facet = sns.FacetGrid(data=study_summary_df, col="metric", height=4, aspect=1.61, sharey=False, col_wrap=3)
facet.map_dataframe(sns.pointplot, x="method", hue="method", y="value", join=False, errorbar=("pi", 95), capsize=0.2)
facet.add_legend();

In [ ]:
df

In [ ]:
data_stabilization_df = (
    df.groupby(["method"])
                    .expanding()
                    .mean()
                    .reset_index()
)
data_stabilization_df["level_1"] = data_stabilization_df.groupby(["method"]).cumcount()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
sns.lineplot(data=data_stabilization_df, x="level_1", hue="method", y="means", ax=ax)
ax.axhline(TRUE_MEAN, linestyle="--", color="dimgrey")

In [ ]:
stabilization_df = (
    study_summary_df.groupby(["method", "metric"])
                    .expanding()
                    .mean()
                    .reset_index()
)
stabilization_df["level_2"] = stabilization_df.groupby(["method", "metric"]).cumcount()

In [ ]:
facet = sns.FacetGrid(
    data=stabilization_df,
    col="metric",
    height=4,
    aspect=1.61,
    sharex=False,
    sharey=False,
    col_wrap=3,
)
facet.map_dataframe(sns.lineplot, x="level_2", hue="method", y="value")
facet.add_legend();